CODE TO MOUNT THE COLAB TO GOOGLE DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

CODE TO SPLIT THE DATA INTO TRAIN VALID AND TEST FOLDERS

In [ ]:
import os
import shutil
import random

# Paths to your dataset
dataset_dir = '/content/drive/MyDrive/tekskills/helmet/data'  # Replace with the path to your folder containing images and labels
output_dir = '/content/drive/MyDrive/tekskills/helmet/output'  # Replace with the path to your output directory

# Paths to store the split dataset
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')
test_dir = os.path.join(output_dir, 'test')

# Create the directories
os.makedirs(os.path.join(train_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'labels'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'labels'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'labels'), exist_ok=True)

# Get all image files (assume YOLO format, where each image has a corresponding .txt file)
all_files = [f for f in os.listdir(dataset_dir) if f.endswith('.jpg') or f.endswith('.png')]
random.shuffle(all_files)

# Define the split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Calculate split indices
total_files = len(all_files)
train_split = int(total_files * train_ratio)
val_split = int(total_files * (train_ratio + val_ratio))

# Split the files into train, val, test
train_files = all_files[:train_split]
val_files = all_files[train_split:val_split]
test_files = all_files[val_split:]

# Helper function to copy image and corresponding label
def copy_files(file_list, dest_dir):
    for file_name in file_list:
        image_path = os.path.join(dataset_dir, file_name)
        label_path = os.path.join(dataset_dir, file_name.replace('.jpg', '.txt').replace('.png', '.txt'))

        # Copy image and corresponding label (if label exists)
        if os.path.exists(label_path):
            shutil.copy(image_path, os.path.join(dest_dir, 'images', file_name))
            shutil.copy(label_path, os.path.join(dest_dir, 'labels', file_name.replace('.jpg', '.txt').replace('.png', '.txt')))

# Copy the files to train, val, and test directories
copy_files(train_files, train_dir)
copy_files(val_files, val_dir)
copy_files(test_files, test_dir)

print(f"Dataset split complete. Train: {len(train_files)}, Val: {len(val_files)}, Test: {len(test_files)}")

Dataset split complete. Train: 964, Val: 121, Test: 121


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.6/913.6 kB 12.5 MB/s eta 0:00:00


YOLO MODEL TRAINING CODE

In [ ]:
from ultralytics import YOLO
import os

# Paths
dataset_dir = "/content/drive/MyDrive/tekskills/helmet/output"  # Path to your dataset folder (containing train, val, and test folders)
classes_file = os.path.join(dataset_dir, "classes.txt")  # Path to your classes.txt file
data_yaml_path = os.path.join(dataset_dir, "data.yaml")  # Output path for the dynamically created data.yaml
pretrained_model = "yolov8m.pt"  # Choose a YOLOv8 model (e.g., yolov8n, yolov8s, yolov8m, etc.)
output_dir = "/content/drive/MyDrive/tekskills/helmet/yolov8_training"  # Directory to save training results
run_name = "custom_model"  # Run name for this training session
epochs = 200  # Number of training epochs
batch_size = 16  # Batch size
img_size = 640  # Image size for training

# Ensure train, val, and test directories exist
train_images = os.path.join(dataset_dir, "train", "images")
val_images = os.path.join(dataset_dir, "val", "images")
test_images = os.path.join(dataset_dir, "test", "images")
if not all([os.path.exists(train_images), os.path.exists(val_images), os.path.exists(test_images)]):
    raise FileNotFoundError("train, val, or test directories not found in the dataset folder.")

# Read classes from the classes.txt file
if not os.path.exists(classes_file):
    raise FileNotFoundError(f"classes.txt not found at {classes_file}")

with open(classes_file, "r") as f:
    class_names = [line.strip() for line in f.readlines()]

# Create the data.yaml file dynamically
with open(data_yaml_path, "w") as f:
    f.write(f"train: {train_images}\n")
    f.write(f"val: {val_images}\n")
    f.write(f"test: {test_images}\n")
    f.write(f"nc: {len(class_names)}\n")
    f.write(f"names: {class_names}\n")

print(f"data.yaml file created at {data_yaml_path}")

# Initialize YOLO model
model = YOLO(pretrained_model)

# Train the model
results = model.train(
    data=data_yaml_path,     # Path to the dataset YAML file
    epochs=epochs,           # Total number of epochs
    batch=batch_size,        # Batch size
    imgsz=img_size,          # Image size
    project=output_dir,      # Save results to the specified project directory
    name=run_name,           # Name for this training session
    workers=4,               # Number of data loading workers
    lr0=1e-3                 # Initial learning rate
)

# Validate the model
metrics = model.val()

# Save the trained model weights in .pt format
trained_model_path = os.path.join(output_dir, run_name, "weights", "best.pt")
if os.path.exists(trained_model_path):
    print(f"Training complete. Trained model saved at: {trained_model_path}")
else:
    print("Training complete, but no trained weights found!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
data.yaml file created at /content/drive/MyDrive/Helmet/output/data.yaml


100%|██████████| 49.7M/49.7M [00:00<00:00, 263MB/s]


Ultralytics 8.3.68 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/Helmet/output/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=yolov8_training, name=custom_model, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_

100%|██████████| 755k/755k [00:00<00:00, 122MB/s]


Overriding model.yaml nc=80 with nc=9

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

...


155 epochs completed in 1.764 hours.
Optimizer stripped from yolov8_training/custom_model/weights/last.pt, 52.1MB
Optimizer stripped from yolov8_training/custom_model/weights/best.pt, 52.1MB

Validating yolov8_training/custom_model/weights/best.pt...
Ultralytics 8.3.68 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25,844,971 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.02it/s]


                   all        121        752      0.821      0.838      0.871      0.503
                helmet        115        254      0.873      0.913      0.952      0.557
             no-helmet          9         17      0.922      0.696      0.821      0.365
               no-vest         47         78      0.639      0.808      0.727      0.387
                person        118        278      0.831      0.932       0.94      0.661
                  vest         65        125       0.84      0.843      0.914      0.547
Speed: 0.5ms preprocess, 9.9ms inference, 0.0ms loss, 5.0ms postprocess per image
Results saved to yolov8_training/custom_model
Ultralytics 8.3.68 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25,844,971 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /content/drive/MyDrive/Helmet/output/val/labels.cache... 121 images, 0 backgrounds, 0 corrupt: 100%|██████████| 121/121 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:07<00:00,  1.05it/s]


                   all        121        752       0.82      0.848      0.872      0.502
                helmet        115        254      0.867      0.927      0.959      0.556
             no-helmet          9         17      0.923      0.701      0.822      0.367
               no-vest         47         78       0.63      0.831      0.724      0.381
                person        118        278      0.833      0.935      0.939      0.661
                  vest         65        125      0.847      0.845      0.915      0.543
Speed: 0.7ms preprocess, 21.0ms inference, 0.0ms loss, 7.0ms postprocess per image
Results saved to yolov8_training/custom_model2
Training complete. Trained model saved at: yolov8_training/custom_model/weights/best.pt


INFERENCE CODE VERSION1

In [ ]:
from ultralytics import YOLO
import cv2

# Path to your pretrained YOLOv8 weights file
model_path = '/content/drive/MyDrive/tekskills/helmet/best.pt'
# Path to the input video
input_video_path = '/content/drive/MyDrive/tekskills/helmet/video.mp4'
# Path to save the output video
output_video_path = '/content/drive/MyDrive/tekskills/helmet/helmet_output5.mp4'

# Load the pretrained YOLOv8 model
model = YOLO(model_path)

# Open the input video
cap = cv2.VideoCapture(input_video_path)
# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object to save the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Process each frame in the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 inference on the frame
    results = model(frame)

    # Draw bounding boxes and labels on the frame
    annotated_frame = results[0].plot()  # Plot method returns the frame with annotations

    # Write the annotated frame to the output video
    out.write(annotated_frame)

# Release resources
cap.release()
out.release()

print(f'Inference completed. Output saved at {output_video_path}')

Streaming output truncated to the last 5000 lines.
Speed: 3.4ms preprocess, 15.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 helmets, 2 persons, 2 vests, 15.0ms
Speed: 4.0ms preprocess, 15.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 helmets, 2 persons, 2 vests, 17.0ms
Speed: 3.8ms preprocess, 17.0ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 helmets, 2 persons, 2 vests, 15.0ms
Speed: 4.4ms preprocess, 15.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 helmets, 2 persons, 2 vests, 15.4ms
Speed: 2.9ms preprocess, 15.4ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 helmets, 2 persons, 2 vests, 17.4ms
Speed: 4.0ms preprocess, 17.4ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 helmets, 2 persons, 2 vests, 17.0ms
Speed: 4.1ms preprocess, 17.0ms inference, 2.0ms postprocess per im

INFERENCE CODE WITH THTRESHOLD VALUE

In [ ]:
from ultralytics import YOLO
import cv2
from collections import defaultdict

# Path to your pretrained YOLOv8 weights file
model_path = '/content/drive/MyDrive/tekskills/helmet/best.pt'
# Path to the input video
input_video_path = '/content/drive/MyDrive/tekskills/helmet/video.mp4'
# Path to save the output video
output_video_path = '/content/drive/MyDrive/tekskills/helmet/helmet_output2.mp4'

# Load the pretrained YOLOv8 model
model = YOLO(model_path)

# Open the input video
cap = cv2.VideoCapture(input_video_path)
# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total number of frames

# Define the codec and create VideoWriter object to save the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Confidence threshold
CONFIDENCE_THRESHOLD = 0.5
# Number of frames a detection must persist
MIN_FRAMES = 5

# Dictionary to track detections across frames
detection_tracker = defaultdict(int)

# Process each frame in the video
current_frame = 0  # Initialize frame counter
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    current_frame += 1  # Increment frame counter

    # Run YOLOv8 inference on the frame
    results = model(frame)
    detections = results[0].boxes.data  # Get bounding box data

    # Filter detections and update the tracker
    filtered_detections = []
    for det in detections:
        x1, y1, x2, y2, conf, cls = det[:6]
        if conf >= CONFIDENCE_THRESHOLD and int(cls) != 0:  # Exclude "person" class (cls == 0)
            object_id = (int(cls), int(x1), int(y1), int(x2), int(y2))  # Use bounding box and class as ID
            detection_tracker[object_id] += 1

            if detection_tracker[object_id] >= MIN_FRAMES:
                filtered_detections.append(det)

    # Draw filtered detections on the frame
    for det in filtered_detections:
        x1, y1, x2, y2, conf, cls = det[:6]
        x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
        label = f"{model.names[int(cls)]} {conf:.2f}"

        # Draw a thicker bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), thickness=3)
        # Put label above the bounding box
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), thickness=2)

    # Write the annotated frame to the output video
    out.write(frame)

    # Print progress
    print(f"Processing frame {current_frame}/{total_frames} ({(current_frame/total_frames)*100:.2f}%)", end="\r")

# Release resources
cap.release()
out.release()

print(f'\nInference completed. Output saved at {output_video_path}')


Streaming output truncated to the last 5000 lines.
Processing frame 2579/4245 (60.75%)
0: 384x640 2 helmets, 2 persons, 2 vests, 958.9ms
Speed: 4.4ms preprocess, 958.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2580/4245 (60.78%)
0: 384x640 2 helmets, 2 persons, 2 vests, 964.4ms
Speed: 6.1ms preprocess, 964.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2581/4245 (60.80%)
0: 384x640 2 helmets, 2 persons, 2 vests, 950.8ms
Speed: 4.5ms preprocess, 950.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2582/4245 (60.82%)
0: 384x640 2 helmets, 2 persons, 2 vests, 943.9ms
Speed: 9.3ms preprocess, 943.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2583/4245 (60.85%)
0: 384x640 2 helmets, 2 persons, 2 vests, 947.8ms
Speed: 7.8ms preprocess, 947.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2584/4245 (6

In [ ]:
INFERENCE CODE TO EXCLUDE PERSON CLASS


In [ ]:
from ultralytics import YOLO
import cv2

# Path to your pretrained YOLOv8 weights file
model_path = '/content/drive/MyDrive/tekskills/helmet/best.pt'
# Path to the input video
input_video_path = '/content/drive/MyDrive/tekskills/helmet/twocases.mp4'
# Path to save the output video
output_video_path = '/content/drive/MyDrive/tekskills/helmet/helmet_output4.mp4'

# Load the pretrained YOLOv8 model
model = YOLO(model_path)

# Open the input video
cap = cv2.VideoCapture(input_video_path)
# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object to save the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Define the class ID for "person" (usually class 0 in COCO dataset)
PERSON_CLASS_ID = 0

# Process each frame in the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 inference on the frame
    results = model(frame)
    detections = results[0]  # Access the first frame's results

    # Create a new list to store filtered detections
    filtered_boxes = []
    filtered_scores = []
    filtered_classes = []

    # Loop through detections and filter out "person" class
    for box in detections.boxes:
        class_id = int(box.cls[0])  # Get class ID for the detection
        if class_id != PERSON_CLASS_ID:  # Only include non-person classes
            filtered_boxes.append(box.xyxy[0].tolist())  # Coordinates (x1, y1, x2, y2)
            filtered_scores.append(float(box.conf[0]))   # Confidence score
            filtered_classes.append(class_id)           # Class ID

    # If there are any filtered detections, annotate the frame
    if filtered_boxes:
        for i, bbox in enumerate(filtered_boxes):
            x1, y1, x2, y2 = map(int, bbox)
            score = filtered_scores[i]
            class_id = filtered_classes[i]

            # Annotate frame with bounding boxes and class labels
            label = f"Class {class_id} {score:.2f}"  # Use class ID for labeling
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Write the processed frame to the output video
    out.write(frame)

# Release resources
cap.release()
out.release()

print(f'Inference completed. Output saved at {output_video_path}')



0: 384x640 5 helmets, 2 no-vests, 5 persons, 26.6ms
Speed: 3.0ms preprocess, 26.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 helmets, 2 no-vests, 5 persons, 25.2ms
Speed: 4.2ms preprocess, 25.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 helmets, 2 no-vests, 5 persons, 25.2ms
Speed: 3.6ms preprocess, 25.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 helmets, 2 no-vests, 5 persons, 25.2ms
Speed: 3.7ms preprocess, 25.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 helmets, 1 no-helmet, 2 no-vests, 5 persons, 25.2ms
Speed: 6.1ms preprocess, 25.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 helmets, 1 no-helmet, 2 no-vests, 5 persons, 25.2ms
Speed: 3.6ms preprocess, 25.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 helmets, 3 no-vests, 5 persons, 25.2ms
Speed: 3.7ms preproce

In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/tekskills/helmet/data.zip', 'r') as zip_ref: zip_ref.extractall('/content/drive/MyDrive/tekskills/helmet')


## ***INFERENCE CODE FILE WITH REQUIRED CLASSES***

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

# Path to your pretrained YOLOv8 weights file
model_path = '/content/drive/MyDrive/tekskills/helmet/best.pt'
# Path to the input video
input_video_path = '/content/drive/MyDrive/tekskills/helmet/no_helmet.mp4'
# Path to save the output video
output_video_path = '/content/drive/MyDrive/tekskills/helmet/helmet_output7.mp4'

# Load the pretrained YOLOv8 model
model = YOLO(model_path)

# Define the classes to detect
classes_to_detect = ['helmet', 'no-helmet', 'vest', 'no-vest']

# Open the input video
cap = cv2.VideoCapture(input_video_path)
# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object to save the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Process each frame in the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 inference on the frame
    results = model(frame)[0]  # Get the first (and only) result

    # Create a list to store filtered detections
    filtered_boxes = []
    filtered_cls = []
    filtered_conf = []

    # Filter detections based on the specified classes
    for box in results.boxes:
        class_name = model.names[int(box.cls[0])]
        if class_name in classes_to_detect:
            filtered_boxes.append(box.xyxy[0].cpu().numpy())
            filtered_cls.append(box.cls[0].cpu().numpy())
            filtered_conf.append(box.conf[0].cpu().numpy())

    # Draw filtered bounding boxes and labels on the frame
    annotated_frame = frame.copy()
    for box, cls_id, conf in zip(filtered_boxes, filtered_cls, filtered_conf):
        x1, y1, x2, y2 = map(int, box)
        class_name = model.names[int(cls_id)]

        # Draw bounding box
        color = (0, 255, 0) if 'no' not in class_name else (0, 0, 255)  # Green for positive, Red for negative
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)

        # Draw label
        label = f'{class_name} {conf:.2f}'
        label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
        cv2.rectangle(annotated_frame, (x1, y1 - label_size[1] - 10), (x1 + label_size[0], y1), color, -1)
        cv2.putText(annotated_frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Write the annotated frame to the output video
    out.write(annotated_frame)

# Release resources
cap.release()
out.release()

print(f'Inference completed. Output saved at {output_video_path}')

Streaming output truncated to the last 5000 lines.
Speed: 4.0ms preprocess, 1110.6ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 helmet, 1138.5ms
Speed: 5.7ms preprocess, 1138.5ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 helmet, 1 vest, 1077.6ms
Speed: 3.7ms preprocess, 1077.6ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 helmet, 1 vest, 703.4ms
Speed: 6.3ms preprocess, 703.4ms inference, 0.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 helmet, 1 person, 1 vest, 723.2ms
Speed: 3.6ms preprocess, 723.2ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 helmet, 1 person, 1 vest, 712.9ms
Speed: 4.2ms preprocess, 712.9ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 helmet, 1 person, 1 vest, 701.4ms
Speed: 4.2ms preprocess, 701.4ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x